[![Abrir no Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/flavioluizseixas/llm-first-steps/blob/main/Ex4.ipynb)

## 🔴 4. Chatbot com RAG a partir de PDF (Perguntas Baseadas em Contexto)

**🔹 Enunciado:**  
Implemente um sistema de RAG (Retrieval-Augmented Generation) com suporte a upload de PDF. O chatbot deve responder perguntas com base no conteúdo extraído do documento.

**🧩 Implementação:**  
1. Extrai texto do PDF usando `PyMuPDF`.
2. Divide o texto em chunks e gera embeddings com `SentenceTransformer`.
3. Usa `FAISS` para indexar os chunks e recuperar o mais relevante à pergunta.
4. Gera resposta contextualizada com base no chunk recuperado.
5. Interface Gradio com `gr.File`, `gr.Chatbot`, histórico e controle de estado.

**🚀 Resultados Esperados:**
- Upload e indexação de PDF textual.
- Perguntas respondidas com base no conteúdo do arquivo.
- Sistema robusto para construção de assistentes contextuais.

In [ ]:
#!pip install gradio PyMuPDF faiss-cpu

In [ ]:
import gradio as gr
import torch
import fitz  # PyMuPDF
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import faiss

In [ ]:
# 1. Carrega modelo de linguagem
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

In [ ]:
# 2. Modelo de embeddings
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
# 3. Variáveis globais
document_chunks = []
index = None
chunk_texts = []

In [ ]:
# 4. Extrai e indexa o PDF
def processar_pdf(pdf):
    global index, chunk_texts, document_chunks

    if pdf is None:
        return "⚠️ Nenhum arquivo PDF foi enviado."

    try:
        # Abre o arquivo diretamente pelo caminho
        doc = fitz.open(pdf.name)
    except Exception as e:
        return f"❌ Erro ao ler o PDF: {str(e)}"
    
    #doc = fitz.open(stream=pdf.read(), filetype="pdf")
    texto_total = ""
    for pagina in doc:
        texto_total += pagina.get_text()

    # Dividir em chunks simples (ajustável)
    document_chunks = [texto_total[i:i+500] for i in range(0, len(texto_total), 500)]
    chunk_texts = document_chunks
    embeddings = embed_model.encode(document_chunks, show_progress_bar=True)

    # Criação do índice FAISS
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    return "PDF processado com sucesso. Agora você pode fazer perguntas."

In [ ]:
# 5. Função de resposta com busca e contexto
def responder(pergunta, historico):
    if index is None:
        return "Por favor, envie um PDF primeiro.", historico

    # Busca: pega top 1 chunk mais similar
    pergunta_embed = embed_model.encode([pergunta])
    distancias, indices = index.search(pergunta_embed, k=1)
    contexto = chunk_texts[indices[0][0]]

    # Monta prompt com contexto
    prompt = f"""<|system|>Você é um assistente útil que responde sempre em português, com base no contexto fornecido.<|end|>
<|user|>Contexto: {contexto}\n\nPergunta: {pergunta}<|end|>
<|assistant|>"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    saida = tokenizer.decode(outputs[0], skip_special_tokens=True)
    resposta = saida.split("<|assistant|>")[-1].strip().split("<|end|>")[0].strip()

    historico.append((pergunta, resposta))
    return "", historico

In [ ]:
# 6. Interface Gradio
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 RAG: Perguntas com base em PDF")
    upload = gr.File(label="Envie um PDF")
    status = gr.Markdown()
    chatbot = gr.Chatbot()
    entrada = gr.Textbox(label="Digite sua pergunta")
    limpar = gr.Button("Limpar")
    estado = gr.State([])

    upload.change(processar_pdf, upload, status)
    entrada.submit(responder, [entrada, estado], [entrada, chatbot])
    limpar.click(lambda: ([], ""), None, [chatbot, estado])

demo.launch()
